# 🔗 LangGraph Agent: Поиск разрешённых программ

Агент который: общается → определяет запрос о программе → ищет в списке → подтверждает → ищет в интернете → генерирует HTML отчёт

## 1. Установка и импорты

In [ ]:
!pip install -q langgraph transformers accelerate ddgs torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.1/413.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# Скрываем предупреждения
import warnings
warnings.filterwarnings("ignore")

# Типизация
from typing import TypedDict, List, Literal, Annotated
import operator
from datetime import datetime

# LangGraph
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# LLM и поиск
from transformers import pipeline
from ddgs import DDGS

print("✅ Импорты загружены")

✅ Импорты загружены


## 2. Конфигурация и загрузка модели

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
PROGRAMS_FILE = "allowed_programs.txt"

print(f"⏳ Загружаем {MODEL_NAME}...")
generator = pipeline("text-generation", model=MODEL_NAME, device_map="auto", torch_dtype="auto")
print("✅ Модель загружена!")

⏳ Загружаем Qwen/Qwen2.5-3B-Instruct...


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


✅ Модель загружена!


## 3. Файл со списком программ

In [ ]:
%%writefile allowed_programs.txt
Visual Studio Code
VSCode
PyCharm
Google Chrome
Mozilla Firefox
Slack
Zoom
Microsoft Teams
Docker
Git
Python
Node.js
Postman
Figma
Notion

Writing allowed_programs.txt


## 4. State агента

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[List[str], operator.add]  # История (накапливается)
    current_input: str              # Текущий ввод
    is_program_request: bool        # Это запрос о программе?
    programs_list: str              # Список программ из файла
    matched_program: str            # Найденная программа
    user_confirmed: bool            # Подтверждено?
    search_results: List[str]       # Результаты поиска
    report_path: str                # Путь к отчёту

## 5. Вспомогательные функции

In [ ]:
def call_llm(prompt: str, max_tokens: int = 500) -> str:
    """Вызов LLM"""
    messages = [{"role": "user", "content": prompt}]
    response = generator(messages, max_new_tokens=max_tokens, do_sample=False)
    return response[0]["generated_text"][-1]["content"]

def search_web(query: str, max_results: int = 5) -> List[dict]:
    """Поиск в интернете"""
    try:
        return DDGS().text(query, max_results=max_results)
    except Exception as e:
        print(f"⚠️ Ошибка поиска: {e}")
        return [{"title": "Ошибка", "body": str(e), "href": ""}]

def load_programs_list(filepath: str) -> str:
    """Загрузка списка программ"""
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            return f.read()
    except:
        return "VSCode\nChrome\nSlack"

## 6. Узлы графа

In [ ]:
def chat_node(state: AgentState) -> dict:
    """Приём сообщения"""
    print(f"💬 Получено: {state['current_input']}")
    return {"messages": [f"User: {state['current_input']}"]}


def router_node(state: AgentState) -> dict:
    """Определение типа запроса"""
    prompt = f"""Запрос: "{state['current_input']}"

Это вопрос о программе/приложении/браузере/редакторе?
Примеры вопросов о программах: "расскажи про VSCode", "что такое Docker", "хочу узнать про Chrome"
Примеры обычных вопросов: "привет", "как дела", "спасибо"

Ответь ОДНИМ словом: PROGRAM или CHAT"""

    response = call_llm(prompt, max_tokens=10).strip().upper()
    is_program = "PROGRAM" in response
    print(f"🤖 Router: {'режим агента' if is_program else 'обычный чат'}")
    return {"is_program_request": is_program}


def regular_chat_node(state: AgentState) -> dict:
    """Обычный чат"""
    response = call_llm(f"Ответь дружелюбно: {state['current_input']}", max_tokens=200)
    print(f"💭 {response}")
    return {"messages": [f"Assistant: {response}"]}


def load_programs_node(state: AgentState) -> dict:
    """Загрузка списка программ"""
    programs = load_programs_list(PROGRAMS_FILE)
    print(f"📄 Загружен список ({len(programs.splitlines())} программ)")
    return {"programs_list": programs}


def search_program_node(state: AgentState) -> dict:
    """Поиск программы в списке"""
    prompt = f"""Список программ:
{state['programs_list']}

Запрос: "{state['current_input']}"

Найди программу из списка. Если нашёл — напиши FOUND: название
Если нет — напиши NOT_FOUND"""

    response = call_llm(prompt, max_tokens=50).strip()
    print(f"🔍 {response}")

    if "FOUND:" in response:
        program = response.split("FOUND:")[1].strip()
        return {"matched_program": program}
    return {"matched_program": ""}


def not_found_node(state: AgentState) -> dict:
    msg = "Не нашёл эту программу в списке разрешённых."
    print(f"❌ {msg}")
    return {"messages": [f"Assistant: {msg}"]}


def ask_confirmation_node(state: AgentState) -> dict:
    msg = f"Нашёл '{state['matched_program']}'. Это то что нужно? (да/нет)"
    print(f"✋ {msg}")
    return {"messages": [f"Assistant: {msg}"]}


def cancelled_node(state: AgentState) -> dict:
    msg = "Жаль. Попробуй уточнить или спроси о другой программе."
    print(f"🚫 {msg}")
    return {"messages": [f"Assistant: {msg}"]}


def web_search_node(state: AgentState) -> dict:
    """Поиск в интернете"""
    program = state["matched_program"]
    print(f"🌐 Ищу информацию о '{program}'...")

    raw = search_web(f"{program} программа описание обзор", max_results=5)
    results = [f"{r.get('title', '')}\n{r.get('body', '')}\n{r.get('href', '')}" for r in raw]

    print(f"   Найдено {len(results)} результатов")
    return {"search_results": results}


def generate_report_node(state: AgentState) -> dict:
    """Генерация HTML отчёта"""
    program = state["matched_program"]
    results = state["search_results"]
    now = datetime.now().strftime("%d.%m.%Y %H:%M")

    results_html = "".join([f'<div class="r"><p>{r.replace(chr(10), "<br>")}</p></div>' for r in results])

    html = f'''<!DOCTYPE html>
<html><head><meta charset="UTF-8"><title>Отчёт: {program}</title>
<style>
body{{font-family:system-ui;max-width:800px;margin:40px auto;padding:20px;background:#f5f5f5}}
.c{{background:#fff;border-radius:12px;padding:30px;box-shadow:0 2px 10px rgba(0,0,0,0.1)}}
h1{{color:#2563eb}}.meta{{color:#666;border-bottom:1px solid #eee;padding-bottom:20px}}
.r{{background:#f8fafc;border:1px solid #e2e8f0;border-radius:8px;padding:15px;margin:10px 0}}
</style></head>
<body><div class="c">
<h1>📋 {program}</h1>
<p class="meta">Дата: {now} | ✓ Разрешена</p>
<h2>Найденная информация</h2>
{results_html}
</div></body></html>'''

    filename = f"report_{program.replace(' ', '_')}.html"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html)

    print(f"📄 Отчёт: {filename}")
    return {"report_path": filename, "messages": [f"Assistant: Отчёт сохранён: {filename}"]}

## 7. Роутеры

In [ ]:
def route_by_type(state: AgentState) -> Literal["program", "chat"]:
    return "program" if state["is_program_request"] else "chat"

def route_by_found(state: AgentState) -> Literal["found", "not_found"]:
    return "found" if state["matched_program"] else "not_found"

def route_by_confirm(state: AgentState) -> Literal["yes", "no"]:
    return "yes" if state["user_confirmed"] else "no"

## 8. Сборка графа

In [ ]:
graph = StateGraph(AgentState)

# Узлы
graph.add_node("chat", chat_node)
graph.add_node("router", router_node)
graph.add_node("regular_chat", regular_chat_node)
graph.add_node("load_programs", load_programs_node)
graph.add_node("search_program", search_program_node)
graph.add_node("not_found", not_found_node)
graph.add_node("ask_confirm", ask_confirmation_node)
graph.add_node("cancelled", cancelled_node)
graph.add_node("web_search", web_search_node)
graph.add_node("report", generate_report_node)

# Рёбра
graph.set_entry_point("chat")
graph.add_edge("chat", "router")
graph.add_conditional_edges("router", route_by_type, {"program": "load_programs", "chat": "regular_chat"})
graph.add_edge("regular_chat", END)
graph.add_edge("load_programs", "search_program")
graph.add_conditional_edges("search_program", route_by_found, {"found": "ask_confirm", "not_found": "not_found"})
graph.add_edge("not_found", END)
graph.add_conditional_edges("ask_confirm", route_by_confirm, {"yes": "web_search", "no": "cancelled"})
graph.add_edge("cancelled", END)
graph.add_edge("web_search", "report")
graph.add_edge("report", END)

# Компиляция
memory = MemorySaver()
app = graph.compile(checkpointer=memory, interrupt_after=["ask_confirm"])

print("✅ Агент готов!")

✅ Агент готов!


## 9. Запуск

In [ ]:
def chat(message: str, thread_id: str = "main"):
    """Отправка сообщения агенту"""
    print("=" * 50)
    print(f"👤 Вы: {message}")
    print("=" * 50)

    state = {
        "messages": [], "current_input": message,
        "is_program_request": False, "programs_list": "",
        "matched_program": "", "user_confirmed": False,
        "search_results": [], "report_path": ""
    }

    config = {"configurable": {"thread_id": thread_id}}
    result = app.invoke(state, config)

    # Если ждём подтверждения
    if result.get("matched_program") and not result.get("user_confirmed"):
        answer = input("\n> (да/нет): ").strip().lower()
        confirmed = answer in ["да", "yes", "y", "д"]
        app.update_state(config, {"user_confirmed": confirmed})
        result = app.invoke(None, config)

    print("\n✅ Готово!")
    return result

## 10. Тесты

In [ ]:
# Обычный чат
chat("Привет!")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


👤 Вы: Привет!
💬 Получено: Привет!
🤖 Router: обычный чат
💭 Привет! Как я могу помочь сегодня?

✅ Готово!


{'messages': ['User: Привет!',
  'Assistant: Привет! Как я могу помочь сегодня?'],
 'current_input': 'Привет!',
 'is_program_request': False,
 'programs_list': '',
 'matched_program': '',
 'user_confirmed': False,
 'search_results': [],
 'report_path': ''}

In [ ]:
# Запрос о программе (введи "да")
chat("Расскажи про VSCode", thread_id="t2")

👤 Вы: Расскажи про VSCode
💬 Получено: Расскажи про VSCode
🤖 Router: режим агента
📄 Загружен список (15 программ)
🔍 FOUND: Visual Studio Code
✋ Нашёл 'Visual Studio Code'. Это то что нужно? (да/нет)

> (да/нет): да
🌐 Ищу информацию о 'Visual Studio Code'...
   Найдено 5 результатов
📄 Отчёт: report_Visual_Studio_Code.html

✅ Готово!


{'messages': ['User: Расскажи про VSCode',
  "Assistant: Нашёл 'Visual Studio Code'. Это то что нужно? (да/нет)",
  'Assistant: Отчёт сохранён: report_Visual_Studio_Code.html'],
 'current_input': 'Расскажи про VSCode',
 'is_program_request': True,
 'programs_list': 'Visual Studio Code\nVSCode\nPyCharm\nGoogle Chrome\nMozilla Firefox\nSlack\nZoom\nMicrosoft Teams\nDocker\nGit\nPython\nNode.js\nPostman\nFigma\nNotion\n',
 'matched_program': 'Visual Studio Code',
 'user_confirmed': True,
 'search_results': ['Что такое Visual Studio Code и как с ним работать\nDec 24, 2024 · Узнайте, что такое Visual Studio Code , как его установить и использовать. Разберем особенности VS Code, его преимущества и отличия от Visual Studio.\nhttps://ru.hexlet.io/blog/posts/chto-takoe-visual-studio-code',
  'Что такое visual studio code (vs code). Обзор программы ...\nРасскажу, что такое visual studio code . Покажу, какие есть основные возможности программы.\nhttps://www.youtube.com/watch?v=YEV0itZ91mk',
  'Vi

In [ ]:
# Программа не в списке
chat("Что такое Photoshop?", thread_id="t3")

👤 Вы: Что такое Photoshop?
💬 Получено: Что такое Photoshop?
🤖 Router: режим агента
📄 Загружен список (15 программ)


## 11. Интерактивный режим

In [ ]:
print("🤖 Напиши 'выход' для остановки")
i = 0
while True:
    msg = input("\n👤 Вы: ").strip()
    if msg.lower() in ["выход", "exit", "q"]: break
    if msg:
        i += 1
        chat(msg, thread_id=f"s{i}")